In [2]:
import trello
import pync
import time
import subprocess
import os
import datetime

def login_and_get_boards(trello):
    wg = trello.members.get('winthropgillis')
    boards = wg['idBoards']
    return boards

def filter_board(trello, b, name):
    board = trello.boards.get(b)
    return board['name'] == name

def get_board(trello, boards, board):
    return [x for x in boards if filter_board(trello, x, board)][0]

def get_lists(trello, b_id):
    lists = trello.boards.get_list(b_id)
    return lists

def get_cards(trello, c_id):
    cards = trello.lists.get_card(c_id)
    return cards

def parseCard(card):
    title = card['name']
    try:
        name, time = title.split('-')
        name = name.strip()
        time = time.strip()
        time, tstr = parseTime(time)
        return (name, time, tstr, title)
    except:
        return tuple([None]*4)

def parseTime(tstr):
    sminute, ssec = tstr.split(' ')
    minute = int(sminute[:-1])
    sec = int(ssec[:-1])
    time_str = ''
    if minute:
        time_str += '{} minutes '.format(minute)
    if sec:
        if time_str:
            time_str += 'and '
        time_str += '{} seconds'.format(sec)
    return (sec+ minute*60, time_str)    


def run_serial_timer(trello, cards, loud=True):
    l = len(cards)
    future, t, tstr, _t = parseCard(cards[0])
    if loud:
        subprocess.call(['say', 'Do {} for {}'.format(future, tstr)])
    for i, card in enumerate(cards):
        if i<l-1:
            future, t, tstr, _t = parseCard(cards[i+1])
        else:
            future = None
        name, t, ttemp, title = parseCard(card)
        trello.cards.new_label(card['id'], 'green')
        minute = t//60
        sec = t%60
        if minute and sec:
            for j in range(minute,0,-1):
                trello.cards.update_name(card['id'], '{} ends in {}'
                                         .format(name, str(j) + ' minutes'))
                time.sleep(60)
            trello.cards.update_name(card['id'], '{} ends in {}'
                                     .format(name, str(sec) + ' seconds'))
            time.sleep(sec)
        elif minute:
            for j in range(minute,1,-1):
                trello.cards.update_name(card['id'], '{} ends in {}'
                                         .format(name, str(j) + ' minutes'))
                time.sleep(60)
            for j in range(50, -1, -10):
                trello.cards.update_name(card['id'], '{} ends in {}'
                                         .format(name, str(j) + ' seconds'))
            
        pync.Notifier.notify('Now do: {}'.format(future), 
                             title='Finished: {}'.format(name))
        if loud:
            subprocess.call(['afplay', 'gong trim.m4a', '-t', '1'])
        if future and loud:
            subprocess.call(['say', '{} is over. Now do {} for {}'
                             .format(name, future, tstr)])
        elif loud:
            subprocess.call(['say', '{} is over'.format(name)])
        trello.cards.delete_label_color('green', card['id'])
        trello.cards.update_name(card['id'], title)

def repeat():
    ser_cards = get_cards(t, serial['id'])
    run_serial_timer(t, ser_cards)
    
def repeat_quiet():
    ser_cards = get_cards(t, serial['id'])
    run_serial_timer(t, ser_cards, False)

def process_tasks(trello, plist):
    plist_cards = get_cards(trello, plist['id'])
    with open('task_history.csv', 'a') as f:
        for card in plist_cards:
            name, t, readable, whole_title = parseCard(card)
            line = ','.join((name,datetime.date.today().strftime('%m-%d-%Y'),str(t)))
            f.write(line + '\n')
            trello.cards.delete(card['id'])


In [3]:
api_key = os.environ['TRELLO_APIKEY']
token = os.environ['TRELLO_TOKEN']

In [4]:
t = trello.TrelloApi(api_key, token=token)

In [4]:
boards = login_and_get_boards(t)

In [5]:
timer_board_id = get_board(t, boards, 'Timers')

In [6]:
lists = get_lists(t, timer_board_id)

In [7]:
serial = list(filter(lambda d: d['name'] == 'Serial', lists))[0]
parallel = list(filter(lambda d: d['name'] == 'Parallel', lists))[0]
processing = list(filter(lambda d: d['name'] == 'For processing', lists))[0]

In [18]:
ser_cards = get_cards(t, serial['id'])

In [10]:
repeat()

In [18]:
# TODO
# Add label to current timer and add to title how long it has left
repeat_quiet()

KeyboardInterrupt: 

In [11]:
process_tasks(t, processing)

In [12]:
serial['id']

'5610135234343b361fb9b094'

In [13]:
with open('serial_id.txt', 'w') as fil:
    fil.write(serial['id'])

In [14]:
with open('timers_id.txt', 'w') as fil:
    fil.write(timer_board_id)

In [19]:
ser_cards[0]

{'badges': {'attachments': 0,
  'checkItems': 0,
  'checkItemsChecked': 0,
  'comments': 0,
  'description': False,
  'due': None,
  'fogbugz': '',
  'subscribed': False,
  'viewingMemberVoted': False,
  'votes': 0},
 'checkItemStates': [],
 'closed': False,
 'dateLastActivity': '2015-10-10T21:22:07.008Z',
 'desc': '',
 'descData': None,
 'due': None,
 'email': 'winthropgillis+54ef444ff6cddfd1ce2be7d3+56197a9d4aa3b90dfe47435c+2b3208322be9b6a61301e11ece8a98118fdc254e@boards.trello.com',
 'id': '56197a9d4aa3b90dfe47435c',
 'idAttachmentCover': None,
 'idBoard': '5610133e828fdc95eded90a5',
 'idChecklists': [],
 'idLabels': ['5610133e19ad3a5dc2a886dd', '5610133e19ad3a5dc2a886da'],
 'idList': '5610135234343b361fb9b094',
 'idMembers': [],
 'idMembersVoted': [],
 'idShort': 100,
 'labels': [{'color': 'blue',
   'id': '5610133e19ad3a5dc2a886dd',
   'idBoard': '5610133e828fdc95eded90a5',
   'name': 'Done',
   'uses': 1},
  {'color': 'orange',
   'id': '5610133e19ad3a5dc2a886da',
   'idBoard': '

In [24]:
t.cards.update(ser_cards[0]['id'])

{'badges': {'attachments': 0,
  'checkItems': 0,
  'checkItemsChecked': 0,
  'comments': 0,
  'description': False,
  'due': None,
  'fogbugz': '',
  'subscribed': False,
  'viewingMemberVoted': False,
  'votes': 0},
 'checkItemStates': [],
 'closed': False,
 'dateLastActivity': '2015-10-10T21:23:55.115Z',
 'desc': '',
 'descData': None,
 'due': None,
 'email': 'winthropgillis+54ef444ff6cddfd1ce2be7d3+56197a9d4aa3b90dfe47435c+2b3208322be9b6a61301e11ece8a98118fdc254e@boards.trello.com',
 'id': '56197a9d4aa3b90dfe47435c',
 'idAttachmentCover': None,
 'idBoard': '5610133e828fdc95eded90a5',
 'idChecklists': [],
 'idLabels': ['5610133e19ad3a5dc2a886da'],
 'idList': '5610135234343b361fb9b094',
 'idMembers': [],
 'idShort': 100,
 'labels': [{'color': 'orange',
   'id': '5610133e19ad3a5dc2a886da',
   'idBoard': '5610133e828fdc95eded90a5',
   'name': 'Not Done',
   'uses': 1}],
 'manualCoverAttachment': False,
 'name': 'test 1 - 0m 15s',
 'pos': 65535,
 'shortUrl': 'https://trello.com/c/1QQtQjR

In [5]:
t.cards.new?


In [8]:
t.cards.new_attachment?